<a href="https://colab.research.google.com/github/sebids10/Speech_Recognition/blob/main/Recunoastere_Vocala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import numpy as np

import tensorflow as tf
from tensorflow import keras

from pathlib import Path
from IPython.display import display, Audio
from google.colab import drive

In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = os.getcwd() + '/drive/My Drive/sample_data/16000_pcm_speeches'
files = os.listdir(path)
for name in files:
    print(name)

tf_Wav_reader.py
audio
noise


In [ ]:
#! sudo apt-get install unrar

In [ ]:
#!unrar e 16000_pcm_speeches.rar

In [ ]:
#!mv /content/sample_data/ /content/drive/MyDrive

In [ ]:
DATASET_ROOT = os.path.join(os.path.expanduser("~"), "/content/drive/MyDrive/sample_data/16000_pcm_speeches")
# The folders in which we will put the audio samples and the noise samples
AUDIO_SUBFOLDER = "audio"
NOISE_SUBFOLDER = "noise"

DATASET_AUDIO_PATH = os.path.join(DATASET_ROOT, AUDIO_SUBFOLDER)
DATASET_NOISE_PATH = os.path.join(DATASET_ROOT, NOISE_SUBFOLDER)

# Percentage of samples to use for validation
VALID_SPLIT = 0.1

# Seed to use when shuffling the dataset and the noise
SHUFFLE_SEED = 43

# The sampling rate to use.
# This is the one used in all of the audio samples.
# We will resample all of the noise to this sampling rate.
# This will also be the output size of the audio wave samples
# (since all samples are of 1 second long)
SAMPLING_RATE = 16000

# The factor to multiply the noise with according to:
#   noisy_sample = sample + noise * prop * scale
#      where prop = sample_amplitude / noise_amplitude
SCALE = 0.5

BATCH_SIZE = 128
EPOCHS = 100

In [ ]:
# If folder `audio`, does not exist, create it, otherwise do nothing
if os.path.exists(DATASET_AUDIO_PATH) is False:
    os.makedirs(DATASET_AUDIO_PATH)

# If folder `noise`, does not exist, create it, otherwise do nothing
if os.path.exists(DATASET_NOISE_PATH) is False:
    os.makedirs(DATASET_NOISE_PATH)

for folder in os.listdir(DATASET_ROOT):
    if os.path.isdir(os.path.join(DATASET_ROOT, folder)):
        if folder in [AUDIO_SUBFOLDER, NOISE_SUBFOLDER]:
            # If folder is `audio` or `noise`, do nothing
            continue
        elif folder in ["other", "_background_noise_"]:
            # If folder is one of the folders that contains noise samples,
            # move it to the `noise` folder
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_NOISE_PATH, folder),
            )
        else:
            # Otherwise, it should be a speaker folder, then move it to
            # `audio` folder
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_AUDIO_PATH, folder),
            )

#Noise preparation

In [ ]:

# Get the list of all noise files
noise_paths = []
for subdir in os.listdir(DATASET_NOISE_PATH):
    subdir_path = Path(DATASET_NOISE_PATH) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

print(
    "Found {} files belonging to {} directories".format(
        len(noise_paths), len(os.listdir(DATASET_NOISE_PATH))
    )
)

Found 6 files belonging to 2 directories


In [ ]:
command = (
    "for dir in `ls -1 " + DATASET_NOISE_PATH + "`; do "
    "for file in `ls -1 " + DATASET_NOISE_PATH + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)

os.system(command)

# Split noise into chunks of 16000 each
def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == SAMPLING_RATE:
        # Number of slices of 16000 each that can be generated from the noise sample
        slices = int(sample.shape[0] / SAMPLING_RATE)
        sample = tf.split(sample[: slices * SAMPLING_RATE], slices)
        return sample
    else:
        print("Sampling rate for {} is incorrect. Ignoring it".format(path))
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

print(
    "{} noise files were split into {} noise samples where each is {} sec. long".format(
        len(noise_paths), noises.shape[0], noises.shape[1] // SAMPLING_RATE
    )
)

6 noise files were split into 354 noise samples where each is 1 sec. long


#Data Generation

In [ ]:
def paths_and_labels_to_dataset(audio_paths, labels):
    """Constructs a dataset of audios and labels."""
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))


def path_to_audio(path):
    """Reads and decodes an audio file."""
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE) # inregistrare pe un canal, stereo(dual-chanel) e pentru calitate mai buna, mono pentru a extrage parametrii corect
    return audio


def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        # Create a random tensor of the same size as audio ranging from
        # 0 to the number of noise stream samples that we have.
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        # Get the amplitude proportion between the audio and the noise
        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        # Adding the rescaled noise to audio
        audio = audio + noise * prop * scale

    return audio


def audio_to_fft(audio):
    # Since tf.signal.fft applies FFT on the innermost dimension,
    # we need to squeeze the dimensions and then expand them again
    # after FFT
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    # Return the absolute value of the first half of the FFT
    # which represents the positive frequencies
    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])


# Get the list of audio file paths along with their corresponding labels

class_names = os.listdir(DATASET_AUDIO_PATH)
print("Our class names: {}".format(class_names,))

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Processing speaker {}".format(name,))
    dir_path = Path(DATASET_AUDIO_PATH) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

print(
    "Found {} files belonging to {} classes.".format(len(audio_paths), len(class_names))
)

# Shuffle
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(audio_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(labels)

# Split into training and validation
num_val_samples = int(VALID_SPLIT * len(audio_paths))
print("Using {} files for training.".format(len(audio_paths) - num_val_samples))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]

print("Using {} files for validation.".format(num_val_samples))
valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

# Create 2 datasets, one for training and the other for validation
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=SHUFFLE_SEED).batch(32)


# Add noise to the training set
train_ds = train_ds.map(
    lambda x, y: (add_noise(x, noises, scale=SCALE), y),
    num_parallel_calls=tf.data.AUTOTUNE,
)

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)

Our class names: ['Benjamin_Netanyau', 'Magaret_Tarcher', 'Jens_Stoltenberg', 'Julia_Gillard', 'Nelson_Mandela', 'Sebi_Deaconu']
Processing speaker Benjamin_Netanyau
Processing speaker Magaret_Tarcher
Processing speaker Jens_Stoltenberg
Processing speaker Julia_Gillard
Processing speaker Nelson_Mandela
Processing speaker Sebi_Deaconu
Found 8317 files belonging to 6 classes.
Using 7486 files for training.
Using 831 files for validation.


#Model Definition

In [ ]:
def residual_block(x, filters, conv_num=3, activation="relu"): # definirea de residual block pentru combaterea gradientului
# sunt folosite pentru a putea invata reprezentari complexe ale datelor
    # Shortcut
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)#residual block facut pentru convolutii
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2)(x)


def build_model(input_shape, num_classes): # sa fie 6 clase(trebuie si cea cu numele meu)
    inputs = keras.layers.Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 2)# 16 filtre 2 straturi convolutionale
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x) # transformam un tensor multidimensional in unul unidimensional, adica il aplatizeaza, pentru preluarea de catre stratul fullyconected, pregatim datele de intrare pentru conectarea cu alte straturi
    x = keras.layers.Dense(256, activation="relu")(x) # straturi dense(fully conected), adica un strat unde toti neuronii sunt conectati
    x = keras.layers.Dense(128, activation="relu")(x) # functia relu este o functie nonliniara care transforma toate valorile negative in 0 si pe cele pozitive le lasa neschimbate

    outputs = keras.layers.Dense(num_classes, activation="softmax", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)


model = build_model((SAMPLING_RATE // 2, 1), len(class_names))

model.summary()

# Compile the model using Adam's default learning rate
model.compile(
    optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Add callbacks:
# 'EarlyStopping' to stop training when the model is not enhancing anymore
# 'ModelCheckPoint' to always keep the model that has the best val_accuracy
model_save_filename = "model_6classes.h5"
save_dir = '/content/drive/MyDrive'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
model_save_path = os.path.join(save_dir, 'model.h5')
earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 8000, 1)]    0           []                               
                                                                                                  
 conv1d_1 (Conv1D)              (None, 8000, 16)     64          ['input[0][0]']                  
                                                                                                  
 activation (Activation)        (None, 8000, 16)     0           ['conv1d_1[0][0]']               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 8000, 16)     784         ['activation[0][0]']             
                                                                                              

#Training

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)
model.save(model_save_path)

Epoch 1/100
59/59 [==============================] - 693s 10s/step - loss: 1.2064 - accuracy: 0.6755 - val_loss: 0.3379 - val_accuracy: 0.8676
Epoch 2/100
59/59 [==============================] - 385s 6s/step - loss: 0.2718 - accuracy: 0.8975 - val_loss: 0.3085 - val_accuracy: 0.8905
Epoch 3/100
59/59 [==============================] - 380s 6s/step - loss: 0.2398 - accuracy: 0.9114 - val_loss: 0.2839 - val_accuracy: 0.9001
Epoch 4/100
59/59 [==============================] - 375s 6s/step - loss: 0.1716 - accuracy: 0.9360 - val_loss: 0.2427 - val_accuracy: 0.9013
Epoch 5/100
59/59 [==============================] - 379s 6s/step - loss: 0.1516 - accuracy: 0.9443 - val_loss: 0.1428 - val_accuracy: 0.9458
Epoch 6/100
59/59 [==============================] - 379s 6s/step - loss: 0.1283 - accuracy: 0.9508 - val_loss: 0.2132 - val_accuracy: 0.9254
Epoch 7/100
59/59 [==============================] - 380s 6s/step - loss: 0.1323 - accuracy: 0.9504 - val_loss: 0.2290 - val_accuracy: 0.9194
Epoch

In [ ]:
!mv model_6classes.h5 /content/drive/MyDrive

In [ ]:
!mv model.h5 /content/drive/MyDrive #modelul antrenat pe mai putine epoch-uri
!mv model.h5_20Epoch /content/drive/MyDrive #modelul antrenat pana la deconectare cu 21 Epoch-uri

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/model_6classes.h5')

#Evaluation

In [ ]:
print(model.evaluate(valid_ds))

26/26 [==============================] - 233s 6s/step - loss: 0.0530 - accuracy: 0.9880
[0.05295250564813614, 0.9879662990570068]


#Testing

In [ ]:
from sklearn.metrics import confusion_matrix
SAMPLES_TO_DISPLAY = 100

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=SCALE), y))

for audios, labels in test_ds.take(1):
    # Get the signal FFT
    ffts = audio_to_fft(audios)
    # Predict
    y_pred = model.predict(ffts)
    # Take random samples
    rnd = np.random.randint(0, BATCH_SIZE, SAMPLES_TO_DISPLAY)
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]
    matrice_confuzie = confusion_matrix(labels, y_pred)

    for index in range(SAMPLES_TO_DISPLAY):
        # For every sample, print the true and predicted label
        # as well as run the voice with the noise
        print(
            "Speaker: {} - Predicted: {}".format(
                class_names[labels[index]],
                class_names[y_pred[index]],
            )
        )
        display(Audio(audios[index, :, :].squeeze(), rate=SAMPLING_RATE))

4/4 [==============================] - 2s 481ms/step
Speaker: Jens_Stoltenberg - Predicted: Jens_Stoltenberg


Speaker: Nelson_Mandela - Predicted: Nelson_Mandela


Speaker: Magaret_Tarcher - Predicted: Magaret_Tarcher


Speaker: Nelson_Mandela - Predicted: Nelson_Mandela


Speaker: Nelson_Mandela - Predicted: Nelson_Mandela


...

Speaker: Benjamin_Netanyau - Predicted: Magaret_Tarcher


Speaker: Sebi_Deaconu - Predicted: Sebi_Deaconu


Speaker: Sebi_Deaconu - Predicted: Sebi_Deaconu


Speaker: Magaret_Tarcher - Predicted: Magaret_Tarcher


Speaker: Magaret_Tarcher - Predicted: Magaret_Tarcher


#Matrice de confuzie


In [ ]:
import pandas as pd
class_names = ['Benjamin_Netanyau', 'Magaret_Tarcher', 'Jens_Stoltenberg', 'Julia_Gillard', 'Nelson_Mandela', 'Sebi_Deaconu']
df_confuzie = pd.DataFrame(matrice_confuzie, index=class_names, columns=class_names)

display(df_confuzie)

,Benjamin_Netanyau,Magaret_Tarcher,Jens_Stoltenberg,Julia_Gillard,Nelson_Mandela,Sebi_Deaconu
Benjamin_Netanyau,24,0,0,0,0,0
Magaret_Tarcher,0,15,0,0,0,0
Jens_Stoltenberg,2,0,28,0,0,0
Julia_Gillard,0,0,0,13,0,0
Nelson_Mandela,0,0,0,0,12,0
Sebi_Deaconu,0,0,0,0,0,6


In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pydub import AudioSegment

audio = AudioSegment.from_wav("Deaconu_Sebi_audio.wav")
audio = audio.set_frame_rate(16000).set_channels(1)
audio.export("Deaconu_sebi_16khz.wav", format="wav")

<_io.BufferedRandom name='Deaconu_sebi_16khz.wav'>

In [ ]:
!mv Deaconu_sebi_16khz.wav /content/drive/MyDrive

In [ ]:
audio = AudioSegment.from_wav("Deaconu_sebi_16khz.wav")
durata = len(audio)
durata_segment = 1000# ms adica o secunda
numar_segmente = durata // durata_segment

for i in range(numar_segmente):
    segment = audio[i*durata_segment:(i+1)*durata_segment]
    segment.export(os.path.join("Folder_audio", f"{i}.wav"), format="wav")

In [ ]:
!mv Sebi_Deaconu /content/drive/MyDrive/sample_data/16000_pcm_speeches/audio